In [4]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [5]:
# Reading the cleaned dataset
df = pd.read_csv('/Users/rohan/Data Science Project/Cleaned_data_salary.csv')

In [6]:
# Choosing variables we would be needing in model selection:

df_model = df[['Size','Type of ownership','Industry','Sector','Revenue','Per hour','Employer Provided','avg_sal',
              'Ratings','Remote', 'Age_Company','Size_greater_1000', 'Seniority', 'State']]


In [7]:
# Converting categorical columns into dummy variables
pd.set_option('display.max_columns', None)
df_dum = pd.get_dummies(df_model)
df_dum

,Per hour,Employer Provided,avg_sal,Ratings,Remote,Age_Company,Size_greater_1000,Size_-1,Size_1 to 50 Employees,Size_1 to 50 employees,Size_10000+ Employees,Size_10000+ employees,Size_1001 to 5000 Employees,Size_1001 to 5000 employees,Size_201 to 500 Employees,Size_201 to 500 employees,Size_5001 to 10000 employees,Size_501 to 1000 Employees,Size_501 to 1000 employees,Size_51 to 200 Employees,Size_51 to 200 employees,Size_Unknown,Type of ownership_-1,Type of ownership_College / University,Type of ownership_Company - Private,Type of ownership_Company - Public,Type of ownership_Government,Type of ownership_Hospital,Type of ownership_Nonprofit Organization,Type of ownership_Other Organization,Type of ownership_School / School District,Type of ownership_Subsidiary or Business Segment,Type of ownership_Unknown,Industry_-1,Industry_Accounting,Industry_Advertising & Marketing,Industry_Aerospace & Defense,Industry_Architectural & Engineering Services,Industry_Auctions & Galleries,Industry_Banks & Credit Unions,Industry_Beauty & Personal Accessories Stores,Industry_Biotech & Pharmaceuticals,Industry_Brokerage Services,Industry_Colleges & Universities,Industry_Computer Hardware & Software,Industry_Construction,Industry_Consulting,Industry_Consumer Product Rental,Industry_Consumer Products Manufacturing,"Industry_Department, Clothing, & Shoe Stores",Industry_Education Training Services,Industry_Energy,Industry_Enterprise Software & Network Solutions,Industry_Farm Support Services,Industry_Federal Agencies,Industry_Financial Analytics & Research,Industry_Financial Transaction Processing,Industry_Food & Beverage Manufacturing,Industry_Gambling,Industry_Gas Stations,Industry_Health Care Products Manufacturing,Industry_Health Care Services & Hospitals,"Industry_Health, Beauty, & Fitness",Industry_IT Services,Industry_Industrial Manufacturing,Industry_Insurance Agencies & Brokerages,Industry_Insurance Carriers,Industry_Internet,Industry_Investment Banking & Asset Management,Industry_K-12 Education,Industry_Lending,Industry_Logistics & Supply Chain,Industry_Metals Brokers,Industry_Mining,Industry_Motion Picture Production & Distribution,Industry_Other Retail Stores,Industry_Real Estate,Industry_Religious Organizations,Industry_Research & Development,Industry_Security Services,Industry_Social Assistance,Industry_Sporting Goods Stores,Industry_Staffing & Outsourcing,Industry_Stock Exchanges,Industry_TV Broadcast & Cable Networks,Industry_Telecommunications Manufacturing,Industry_Telecommunications Services,Industry_Transportation Equipment Manufacturing,Industry_Transportation Management,Industry_Travel Agencies,Industry_Trucking,Industry_Video Games,Industry_Wholesale,Sector_-1,Sector_Accounting & Legal,Sector_Aerospace & Defense,Sector_Agriculture & Forestry,"Sector_Arts, Entertainment & Recreation",Sector_Biotech & Pharmaceuticals,Sector_Business Services,"Sector_Construction, Repair & Maintenance",Sector_Consumer Services,Sector_Education,Sector_Finance,Sector_Government,Sector_Health Care,Sector_Information Technology,Sector_Insurance,Sector_Manufacturing,Sector_Media,Sector_Mining & Metals,Sector_Non-Profit,"Sector_Oil, Gas, Energy & Utilities",Sector_Real Estate,Sector_Retail,Sector_Telecommunications,Sector_Transportation & Logistics,Sector_Travel & Tourism,Revenue_$1 to $2 billion (USD),Revenue_$1 to $5 million (USD),Revenue_$10 to $25 million (USD),Revenue_$10+ billion (USD),Revenue_$100 to $500 million (USD),Revenue_$2 to $5 billion (USD),Revenue_$25 to $50 million (USD),Revenue_$5 to $10 billion (USD),Revenue_$5 to $10 million (USD),Revenue_$50 to $100 million (USD),Revenue_$500 million to $1 billion (USD),Revenue_-1,Revenue_Less than $1 million (USD),Revenue_Unknown / Non-Applicable,Seniority_Jr,Seniority_Sr,Seniority_na,State_AL,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_MI,State_MN,State_MO,State_NC,State_NE,State_NJ,

1. Multiple linear regression - Calculate metrics. Also look for CV
2. Lasso and Ridge regression - Understanding and implementation - Document
3. Decision trees/Random forest regression - Understanding Tuning, how to use models and theory in brief

In [8]:
#Defining a function to calculate the mean absolute error for different models:
    
def calculate_mae(y_true, y_pred, method):
    from sklearn import metrics
    result = metrics.mean_absolute_error(y_true, y_pred)
    print('Mean absolute error for {} is {}'.format(method, result))

1. Multiple linear regression

In [9]:
X = df_dum.drop('avg_sal', axis=1)
y = df_dum['avg_sal'].values

# Splitting dataset into training and testing:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.7, random_state=42)

import statsmodels.api as sm
X_sm = sm.add_constant(X_train)
X_sm_test = sm.add_constant(X_test)
model = sm.OLS(y_train,X_sm)
results = model.fit()
results.summary() # Analyzing weights of variables

# Making predictions and calculating scores:

y_pred_lm = results.predict(X_sm_test)
calculate_mae(y_test, y_pred_lm, 'Multiple Linear Regression')


Mean absolute error for Multiple Linear Regression is 21.62009358634215


Interpretations:

R-squared value for the model is 0.614 which determines that 61% of the variation in the average salary is predicted by the dependent variables. However, not all variables would be significant in explaining the variation in the average salary. Variables with a p value of less than 0.05 could be considered significant.

- A per hour salary would naturally result in a lower average salary. 
- Similarly, if the salary is provided by employer, it is comparatively more than the one estimated by Glassdoors. - If the company has higher ratings, it is bound to provide high average salary. This essentially means that salary is the predominant factor in driving the ratings for a company.
- Internet and Advertising industries comparatively tend to provide higher salaries with an increase in around 20K if all the other variables are kept constant.
- It is surprising to note that revenue of a company does not have a significant role in its average salary.
- Senior positions offer a higher pay with an increase in as much as 19K.
- If you land up a job in California, it would pay around 26K more.

This model provides a great deal of information on how various factors provide an insight in determining the salary for a data scientist position.However, this model may contain multi-collinearity which has not been accounted for. 

Regularized regressions can be applied to pick the most important variables and then check the model score.

Lasso regression:

Using cross validation, optimum value of lambda would be chosen. Optimum lambda value would turn some of the irrelevant coefficients to 0. Predictions would then be peformed using the calculated lambda value.


In [10]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn import metrics

# Setting up parameters for cross validation:
cv = RepeatedKFold(n_repeats=3,n_splits=10,random_state=1)
model_lasso = LassoCV(alphas= np.arange(0,1,0.01), cv=cv, n_jobs=-1)
model_lasso.fit(X_train,y_train)
#model_lasso.alpha_   # Get the optimum value of lambda through cross validation

# Predictions:
y_pred_lasso = model_lasso.predict(X_test)
calculate_mae(y_test, y_pred_lasso, 'Lasso Regression')


/Users/rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/r

Mean absolute error for Lasso Regression is 21.292237518799052


/Users/rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/r

The cross validation chose alpha value as 0.09. Lasso regression turned some of the co-efficients to 0. Using this value of lambda, predictions were performed on the test data. The mean absolute error turned out to be 21.3

Like lasso regression even ridge regression applies the penalty parameter to reduce the coefficients of non contributing coefficients. However, ridge regression would never turn the coefficients to 0. It would be very small but not 0.

In [11]:
# Ridge reression: (Used same CV and alpha range for ridge)

from sklearn.linear_model import RidgeCV
model_ridge = RidgeCV(alphas= np.arange(0,1,0.01), cv=cv)
model_ridge.fit(X_train, y_train)
y_pred_ridge = model_ridge.predict(X_test)

calculate_mae(y_test, y_pred_ridge, 'Ridge regression')

Mean absolute error for Ridge regression is 21.17534254151229


In [12]:
from sklearn.ensemble import RandomForestRegressor

In [13]:
# Decision trees and random forest regression with hypertuning:

rf = RandomForestRegressor(criterion='mae',random_state=1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

calculate_mae(y_test, y_rf, 'Random Forest Regression - test')

Mean absolute error for Random Forest Regression - test is 15.354364406779661


Without tuning the random forest regressor, the mae was found to be 15.35 which is far better than the previous 3 regressions. However, there are various parameters that need to be adjusted for getting the best results from a random forest regressor. Using GridSearchCV we will try to optimize the random forest regression model and verify the mae scores.

In [14]:
#GridSearchCV implementation
# No of features is chosen as the square root of the total no of features. Values are then chosen +-2 around this
# value

from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators":[100,200,300],
    "max_depth":[10, 50, 100],
    "max_features":[11,13,15,17,19]
}

rf_reg_GS = RandomForestRegressor()
rf_tuned = GridSearchCV(estimator=rf_reg_GS, param_grid=param_grid, scoring='ne')

In [38]:
#rf_tuned.fit(X_train, y_train)
#y_pred_rf_tuned = rf_tuned.predict(X_test)
rf_tuned.best_estimator_
#calculate_mae(y_test, y_pred_rf_tuned, 'Random forest tuned')

RandomForestRegressor(max_depth=50, max_features=13, n_estimators=200)

In [28]:
# Decision trees:

from sklearn.tree import DecisionTreeRegressor

parameters = {"splitter": ['best','random'],
              "max_depth": [1,3,5,7,9],
              "max_features":["auto","log2","sqrt",None],
              "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10]}

dec_tree = DecisionTreeRegressor()
dec_tree_tuned = GridSearchCV(estimator=dec_tree, param_grid=parameters, cv=3)

In [34]:
#dec_tree_tuned.fit(X_train, y_train)
y_pred_dt_tuned = dec_tree_tuned.predict(X_test)
calculate_mae(y_test, y_pred_dt_tuned, 'Decision Trees')

Mean absolute error for Decision Trees is 20.155593852468137


Conclusions:
    
Multiple linear regression, lasso and ridge regressions were applied to the dataset to predict the salary of a data scientist. As per the scores reported for each model, random forest regression performed the best with a mean absolute error of 14.91. This means a random forest regression model can predict the salary of a data scientist with an error of +-14.9K $.

These models were tuned using GridSearchCV and other cross-validation techniques. 
The random forest regression model which gave the best results had the following parameters:
Maximum depth = 50
Maximum features = 13
No of trees = 200